In [7]:
from langchain.chat_models import ChatOllama
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.document_loaders import UnstructuredFileLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import OllamaEmbeddings, CacheBackedEmbeddings
from langchain.vectorstores import FAISS
from langchain.storage import LocalFileStore
from langchain.prompts import ChatPromptTemplate
from langchain.schema.runnable import RunnablePassthrough

In [4]:

ollama = ChatOllama(
    model = "llama2:7b",
    temperature=0.1,
    streaming=True,
    callbacks=[StreamingStdOutCallbackHandler()]
)

cache_dir = LocalFileStore("./.cache/")

splitter = CharacterTextSplitter.from_tiktoken_encoder(
    separator="\n",
    chunk_size=600,
    chunk_overlap=100,
)
loader = UnstructuredFileLoader("./files/Chaptor One.txt")

docs = loader.load_and_split(text_splitter=splitter)

embeddings = OllamaEmbeddings()

cached_embeddings = CacheBackedEmbeddings.from_bytes_store(embeddings, cache_dir)

vectorstore = FAISS.from_documents(docs, cached_embeddings)

retriver = vectorstore.as_retriever()


In [5]:
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant. Answer questions using only the following context. If you don't know the answer just say you don't know, don't make it up:\n\n{context}"),
    ("human", "{question}"),
])

In [8]:
chain = {"context": retriver, "question": RunnablePassthrough(),} | prompt | ollama

In [9]:
chain.invoke("Where does Winston live")

Winston Smith lives in London, in a small, cramped flat located in the "Victory Mansions" building. The exact address of his flat is not specified in the novel, but it is mentioned that it is located in the "Airstrip One" district of London, which is presumably a fictional representation of the city.

Winston's flat is described as being in a state of disrepair and lacking many of the modern conveniences that are available in other parts of the world. It has a small kitchen area with a sink, a stove, and a refrigerator, as well as a bedroom with a double bed and a small table for eating meals. The flat is also equipped with a telescreen, which is a two-way communication device that allows the Thought Police to monitor Winston's activities and thoughts at any time.

Overall, Winston's living situation reflects his relatively low social status within the Party hierarchy and the oppressive nature of the totalitarian regime he lives under.

AIMessage(content='Winston Smith lives in London, in a small, cramped flat located in the "Victory Mansions" building. The exact address of his flat is not specified in the novel, but it is mentioned that it is located in the "Airstrip One" district of London, which is presumably a fictional representation of the city.\n\nWinston\'s flat is described as being in a state of disrepair and lacking many of the modern conveniences that are available in other parts of the world. It has a small kitchen area with a sink, a stove, and a refrigerator, as well as a bedroom with a double bed and a small table for eating meals. The flat is also equipped with a telescreen, which is a two-way communication device that allows the Thought Police to monitor Winston\'s activities and thoughts at any time.\n\nOverall, Winston\'s living situation reflects his relatively low social status within the Party hierarchy and the oppressive nature of the totalitarian regime he lives under.')

In [ ]:
chain.run("Describe Vectory Mension")